## Part 01 - EDA with Pyspark

Gradient Boosted Trees applied to Fraud detection

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, countDistinct
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, array, lit
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier

In [2]:
spark = SparkSession.builder.appName('FraudTreeMethods').getOrCreate()

## Read Data

In [53]:
# Load and parse the data file, converting it to a DataFrame.
#data = sqlContext.sql("SELECT * FROM fraud_train_sample_csv")
data = spark.read.csv('train_sample.csv', inferSchema=True, header=True)

In [54]:
data.show(5)

+------+---+------+---+-------+-------------------+---------------+-------------+
|    ip|app|device| os|channel|         click_time|attributed_time|is_attributed|
+------+---+------+---+-------+-------------------+---------------+-------------+
| 87540| 12|     1| 13|    497|2017-11-07 09:30:38|           null|            0|
|105560| 25|     1| 17|    259|2017-11-07 13:40:27|           null|            0|
|101424| 12|     1| 19|    212|2017-11-07 18:05:24|           null|            0|
| 94584| 13|     1| 13|    477|2017-11-07 04:58:08|           null|            0|
| 68413| 12|     1|  1|    178|2017-11-09 09:00:09|           null|            0|
+------+---+------+---+-------+-------------------+---------------+-------------+
only showing top 5 rows



## Convert the click time to day and hour and add it to data.

In [55]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth

In [56]:
from pyspark.sql.functions import hour, minute, dayofmonth
data = data.withColumn('hour',hour(data.click_time)).\
             withColumn('day',dayofmonth(data.click_time))

data.show(5)

+------+---+------+---+-------+-------------------+---------------+-------------+----+---+
|    ip|app|device| os|channel|         click_time|attributed_time|is_attributed|hour|day|
+------+---+------+---+-------+-------------------+---------------+-------------+----+---+
| 87540| 12|     1| 13|    497|2017-11-07 09:30:38|           null|            0|   9|  7|
|105560| 25|     1| 17|    259|2017-11-07 13:40:27|           null|            0|  13|  7|
|101424| 12|     1| 19|    212|2017-11-07 18:05:24|           null|            0|  18|  7|
| 94584| 13|     1| 13|    477|2017-11-07 04:58:08|           null|            0|   4|  7|
| 68413| 12|     1|  1|    178|2017-11-09 09:00:09|           null|            0|   9|  9|
+------+---+------+---+-------+-------------------+---------------+-------------+----+---+
only showing top 5 rows



## Feathering

Feathering, grouping-merging as follow.

In [57]:
gp = data.select("ip","day","hour", "channel")\
               .groupBy("ip","day","hour")\
               .agg({"channel":"count"})\
               .withColumnRenamed("count(channel)", "*ip_day_hour_count_channel")\
               .sort(col("ip"))
data = data.join(gp, ["ip","day","hour"])\
         .sort(col("ip"))

In python EDA we did following:
```python
gp = df[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].\
            count().reset_index().\
            rename(index=str, columns={'channel': '*ip_app_count_channel'})
df = df.merge(gp, on=['ip','app'], how='left')

```
We translate it to Pyspark as follow.

In [58]:
gp = data.select("ip","app", "channel")\
               .groupBy("ip","app")\
               .agg({"channel":"count"})\
               .withColumnRenamed("count(channel)", "*ip_app_count_channel")\
               .sort(col("ip"))
data = data.join(gp, ["ip","app"])\
         .sort(col("ip"))

In python EDA we did following:
```python
gp = df[['ip','app', 'os', 'channel']].\
            groupby(by=['ip', 'app', 'os'])[['channel']].\
            count().reset_index().\
            rename(index=str, columns={'channel': '*ip_app_os_count_channel'})
df = df.merge(gp, on=['ip','app', 'os'], how='left')

```
We translate it to Pyspark as follow.

In [59]:
gp = data.select('ip','app', 'os', 'channel')\
               .groupBy('ip', 'app', 'os')\
               .agg({"channel":"count"})\
               .withColumnRenamed("count(channel)", "*ip_app_os_count_channel")\
               .sort(col("ip"))
data = data.join(gp, ['ip','app', 'os'])\
         .sort(col("ip"))

In python EDA we did following:
```python
gp = df[['ip','day','hour','channel']].\
            groupby(by=['ip','day','channel'])[['hour']].\
            var().reset_index().\
            rename(index=str, columns={'hour': '*ip_tchan_count'})
df = df.merge(gp, on=['ip','day','channel'], how='left')

```
We translate it to Pyspark as follow.

In [63]:
gp = data.select('ip','day','hour','channel')\
               .groupBy('ip','day','channel')\
               .agg({"hour":"variance"})\
               .withColumnRenamed("variance(hour)", "*ip_day_chan_var_hour")\
               .sort(col("ip"))

In [67]:
gp.describe().show()

+-------+-----------------+------------------+------------------+---------------------+
|summary|               ip|               day|           channel|*ip_day_chan_var_hour|
+-------+-----------------+------------------+------------------+---------------------+
|  count|            93176|             93176|             93176|                93176|
|   mean|92922.74640465356| 7.858386279728686| 270.4198828024384|                  NaN|
| stddev|  70800.947593547|0.8932649462156179|130.69830554337338|                  NaN|
|    min|                9|                 6|                 3|                  0.0|
|    max|           364757|                 9|               498|                  NaN|
+-------+-----------------+------------------+------------------+---------------------+



Check out the number of nan and null in the gp.

In [70]:
from pyspark.sql.functions import isnan, when, count, col
gp.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in gp.columns]).show()

+---+---+-------+---------------------+
| ip|day|channel|*ip_day_chan_var_hour|
+---+---+-------+---------------------+
|  0|  0|      0|                89123|
+---+---+-------+---------------------+



We remeber from python EDA the following 

```python
ip                                0
app                               0
device                            0
os                                0
channel                           0
click_time                        0
is_attributed                     0
hour                              0
day                               0
*ip_day_hour_count_channel        0
*ip_app_count_channel             0
*ip_app_os_count_channel          0
*ip_tchan_count               89123
*ip_app_os_var                89715
*ip_app_channel_var_day       84834
*ip_app_channel_mean_hour         0
dtype: int64

```
Therefore we skip the following grouping (columns)as follow.

```python
*ip_tchan_count               10877 non-null float64
*ip_app_os_var                10285 non-null float64
*ip_app_channel_var_day       15166 non-null float64

```
Note that the last gp was not joined into the data.

In python EDA we did following:
```python
gp = df[['ip','app', 'channel','hour']].\
            groupby(by=['ip', 'app', 'channel'])[['hour']].\
            mean().reset_index().\
            rename(index=str, columns={'hour': '*ip_app_channel_mean_hour'})

df = df.merge(gp, on=['ip','app', 'channel'], how='left')

```
We translate it to Pyspark as follow.

In [72]:
gp = data.select('ip','app', 'channel','hour')\
               .groupBy('ip', 'app', 'channel')\
               .agg({"hour":"mean"})\
               .withColumnRenamed("mean(hour)", "*ip_app_channel_mean_hour")\
               .sort(col("ip"))
data = data.join(gp, ['ip', 'app', 'channel'])\
         .sort(col("ip"))

In [73]:
data.printSchema()

root
 |-- ip: integer (nullable = true)
 |-- app: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- os: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- device: integer (nullable = true)
 |-- click_time: string (nullable = true)
 |-- attributed_time: string (nullable = true)
 |-- is_attributed: integer (nullable = true)
 |-- *ip_day_hour_count_channel: long (nullable = false)
 |-- *ip_app_count_channel: long (nullable = false)
 |-- *ip_app_os_count_channel: long (nullable = false)
 |-- avg(hour): double (nullable = true)



In [85]:
# data.summary().show()
cols1 = ['ip', 'app', 'channel',
       'os', 'day', 'hour']
data.describe(cols1).show()

+-------+-----------------+-----------------+------------------+------------------+------------------+-----------------+
|summary|               ip|              app|           channel|                os|               day|             hour|
+-------+-----------------+-----------------+------------------+------------------+------------------+-----------------+
|  count|           100000|           100000|            100000|            100000|            100000|           100000|
|   mean|      91255.87967|         12.04788|         268.83246|          22.81828|           7.86146|          9.32859|
| stddev|69835.55366125253|14.94149992436502|129.72424821194426|55.943135898751194|0.8892663135111792|6.180585781971322|
|    min|                9|                1|                 3|                 0|                 6|                0|
|    max|           364757|              551|               498|               866|                 9|               23|
+-------+-----------------+-----

In [86]:
cols2 = ['device', 'click_time', 
        'attributed_time','is_attributed']
data.describe(cols2).show()

+-------+------------------+-------------------+-------------------+--------------------+
|summary|            device|         click_time|    attributed_time|       is_attributed|
+-------+------------------+-------------------+-------------------+--------------------+
|  count|            100000|             100000|                227|              100000|
|   mean|          21.77125|               null|               null|             0.00227|
| stddev|259.66776742008614|               null|               null|0.047590647702016924|
|    min|                 0|2017-11-06 16:00:00|2017-11-06 17:19:04|                   0|
|    max|              3867|2017-11-09 15:59:51|2017-11-09 15:28:15|                   1|
+-------+------------------+-------------------+-------------------+--------------------+



In [87]:
cols3 = ['*ip_day_hour_count_channel',
       '*ip_app_count_channel',
       '*ip_app_os_count_channel']
data.describe(cols3).show()

+-------+--------------------------+---------------------+------------------------+
|summary|*ip_day_hour_count_channel|*ip_app_count_channel|*ip_app_os_count_channel|
+-------+--------------------------+---------------------+------------------------+
|  count|                    100000|               100000|                  100000|
|   mean|                   1.49328|              3.58026|                 1.29488|
| stddev|        2.0205929005014096|   10.553763885539677|      1.6443882831400434|
|    min|                         1|                    1|                       1|
|    max|                        28|                  132|                      33|
+-------+--------------------------+---------------------+------------------------+



Check out the uniques number for each column in data.

In [93]:
from pyspark.sql.functions import col, countDistinct
data.agg(*(countDistinct(col(c)).alias(c) for c in data.columns)).show()

+-----+---+-------+---+---+----+------+----------+---------------+-------------+--------------------------+---------------------+------------------------+---------+
|   ip|app|channel| os|day|hour|device|click_time|attributed_time|is_attributed|*ip_day_hour_count_channel|*ip_app_count_channel|*ip_app_os_count_channel|avg(hour)|
+-----+---+-------+---+---+----+------+----------+---------------+-------------+--------------------------+---------------------+------------------------+---------+
|34857|161|    161|130|  4|  24|   100|     80350|            227|            2|                        27|                   58|                      23|      324|
+-----+---+-------+---+---+----+------+----------+---------------+-------------+--------------------------+---------------------+------------------------+---------+



In [3]:


# over sampling
major_df = data.filter(col("is_attributed") == 0)
minor_df = data.filter(col("is_attributed") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))
a = range(ratio)

# duplicate the minority rows
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')

# combine both oversampled minority rows and previous majority rows combined_df = major_df.unionAll(oversampled_df)
combined_df = major_df.unionAll(oversampled_df)
combined_df.show()
data = combined_df
data = data.drop('click_time','attributed_time')

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])
assembler = VectorAssembler(inputCols=['ip', 'app', 'device', 'os', 'channel'],outputCol="features")
trainingData = assembler.transform(trainingData)
testData = assembler.transform(testData)

ratio: 439
+------+---+------+---+-------+-------------------+---------------+-------------+
|    ip|app|device| os|channel|         click_time|attributed_time|is_attributed|
+------+---+------+---+-------+-------------------+---------------+-------------+
| 87540| 12|     1| 13|    497|2017-11-07 09:30:38|           null|            0|
|105560| 25|     1| 17|    259|2017-11-07 13:40:27|           null|            0|
|101424| 12|     1| 19|    212|2017-11-07 18:05:24|           null|            0|
| 94584| 13|     1| 13|    477|2017-11-07 04:58:08|           null|            0|
| 68413| 12|     1|  1|    178|2017-11-09 09:00:09|           null|            0|
| 93663|  3|     1| 17|    115|2017-11-09 01:22:13|           null|            0|
| 17059|  1|     1| 17|    135|2017-11-09 01:17:58|           null|            0|
|121505|  9|     1| 25|    442|2017-11-07 10:01:53|           null|            0|
|192967|  2|     2| 22|    364|2017-11-08 09:35:17|           null|            0|
|1436

## Train the model

In [4]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="is_attributed", featuresCol="features", maxIter=20, maxDepth=4)

# Train model.  This also runs the indexers.
model = gbt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "is_attributed", "features").show(5)

+----------+-------------+--------------------+
|prediction|is_attributed|            features|
+----------+-------------+--------------------+
|       0.0|            0|[10.0,12.0,1.0,19...|
|       0.0|            0|[20.0,2.0,1.0,9.0...|
|       0.0|            0|[20.0,2.0,1.0,16....|
|       0.0|            0|[20.0,12.0,1.0,13...|
|       0.0|            0|[20.0,18.0,1.0,19...|
+----------+-------------+--------------------+
only showing top 5 rows



In [5]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="is_attributed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test accuracy = %g" % (accuracy))

Test Error = 0.0536021
Test accuracy = 0.946398


In [6]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|31620|
|       1.0|28359|
+----------+-----+



# Apply to test, predict

In [7]:
test = spark.read.csv('test.csv', inferSchema=True, header=True)
#test.show(5)

assembler = VectorAssembler(inputCols=['ip', 'app', 'device', 'os', 'channel'],outputCol="features")
test = assembler.transform(test)
#test.show(3)

predictions = model.transform(test)
#predictions.show(2)

data_to_submit = predictions.select(['click_id','prediction'])
data_to_submit.show(3)

data_to_submit = data_to_submit.withColumnRenamed('prediction','is_attributed')
data_to_submit.show(3)

data_to_submit.groupBy('is_attributed').count().show()

+--------+----------+
|click_id|prediction|
+--------+----------+
|       0|       1.0|
|       1|       0.0|
|       2|       0.0|
+--------+----------+
only showing top 3 rows

+--------+-------------+
|click_id|is_attributed|
+--------+-------------+
|       0|          1.0|
|       1|          0.0|
|       2|          0.0|
+--------+-------------+
only showing top 3 rows

+-------------+--------+
|is_attributed|   count|
+-------------+--------+
|          0.0|18152789|
|          1.0|  637680|
+-------------+--------+



In [8]:
print('it is runing now')

it is runing now
